Imports

In [ ]:
# !pip install pandas
import pandas as pd

VARS

In [ ]:
csv_file_path = 'input.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
# df.head()
# df.info()


# Duplicates ----------
print("Number of duplicate rows:", len(df[df.duplicated('account_url', keep=False)]))
df[df.duplicated('account_url', keep=False)].sort_values(['account_url'])

---

In [ ]:
def consolidate_rows(rows):
    """Consolidate rows with user input for conflict resolution."""
    consolidated = rows.iloc[0].copy()  # Start with the first row as the base
    for column in rows.columns:
        unique_values = rows[column].dropna().unique()
        if len(unique_values) > 1:
            print(f"\nConflict in '{column}' column with values: {unique_values}")
            # Display the DataFrame with the conflicting values for better context
            display(rows[[column]])  # Using display() for Jupyter Notebook
            chosen_value = input(f"Choose value to keep for {column} (enter the value exactly as shown, or type 'exit' to stop): ")
            if chosen_value.lower() == 'exit':
                return None  # Return None to indicate the user wants to exit
            consolidated[column] = chosen_value
        elif len(unique_values) == 1:
            consolidated[column] = unique_values[0]
    return consolidated

def main(csv_file_path):
    df = pd.read_csv(csv_file_path)
    duplicates = df[df.duplicated('account_url', keep=False)]
    
    if duplicates.empty:
        print("No duplicates found.")
        return df  # Early return if no duplicates
    
    consolidated_list = []
    print("\nStarting the consolidation process...\n")
    for account_url, group in duplicates.groupby('account_url'):
        print(f"Duplicate group for account_url: {account_url}")
        # Print the entire DataFrame for the current duplicate group
        display(group)  # Using display() for Jupyter Notebook
        consolidated_row = consolidate_rows(group)
        if consolidated_row is None:  # User chose to exit
            print("Operation stopped by user.")
            return None
        consolidated_list.append(consolidated_row)
    
    consolidated_df = pd.DataFrame(consolidated_list, columns=df.columns)
    
    # Remove duplicates from original df and append consolidated rows
    df = df.drop_duplicates('account_url', keep=False)
    df = pd.concat([df, consolidated_df], ignore_index=True)
    
    return df

# Example usage
df_processed = main(csv_file_path)
if df_processed is not None:
    # Save the processed DataFrame to 'result.csv'
    df_processed.to_csv('result.csv', index=False)
    print("Edits saved to result.csv.")
else:
    print("Operation was stopped. No changes were saved.")